# Подготовка данных о 3-х комнатных квартирах

Для исследования были собраны объявления с сайта Циан, с использованием самописного парсера. Данные представляют из себя объявления о 3-х комнатных квартирах, во вторичном и многоэтажном жилье, за октябрь 2023 года, в городе Омск.

Для проведения исследования подготовим данные, а именно очистим и преобразуем их.

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_json("../data/data_about_apartments.json")
df.head()

,seller,title,another_title,address,price,price_per_square_meterice
0,Обмен.рф - Брусника Обмен,"3-комн. кв., 66,8 м², 7/10 этаж",None,"Омская область, Омск, Кировский, мкр. 12-й, ул...",6 500 000 ₽,97 305 ₽/м²
1,Обмен.рф - Брусника Обмен,"3-комн. кв., 71,1 м², 1/10 этаж",None,"Омская область, Омск, Кировский, мкр. Прибрежн...",7 700 000 ₽,108 298 ₽/м²
2,КВАРТИРНОЕ РЕШЕНИЕ,"3-комн. кв., 61 м², 1/9 этаж",None,"Омская область, Омск, Советский, мкр. Заозерны...",5 890 000 ₽,96 557 ₽/м²
3,ID 70295415,"3-комн. кв., 62,1 м², 9/9 этаж",None,"Омская область, Омск, Октябрьский, улица Свобо...",4 495 000 ₽,72 383 ₽/м²
4,ID 79765445,"3-комн. кв., 76,8 м², 3/10 этаж",None,"Омская область, Омск, Ленинский, проспект Карл...",ЖК «Граф»,161 992 ₽/м²


In [3]:
df.shape

(1640, 6)

## Очистка данных

In [4]:
# Удаляем дубликаты
df.drop_duplicates(inplace=True)

In [5]:
df.shape

(1433, 6)

In [6]:
# Удаляем квартиры, которые еще не сданы
mask = df.title.str.lower().str.contains("сдача")
df.drop(df[mask].index, inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1432 entries, 0 to 1526
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   seller                     1432 non-null   object
 1   title                      1432 non-null   object
 2   another_title              11 non-null     object
 3   address                    1432 non-null   object
 4   price                      1432 non-null   object
 5   price_per_square_meterice  1432 non-null   object
dtypes: object(6)
memory usage: 78.3+ KB


## Преобразование данных

In [8]:
def find_square_flat(x: str) -> float:
    """
    Получаем площадь квартиры
    """
    pattern = r", (\d+.\d+ м²|\d+. м²)"
    find_square = re.findall(pattern, x)[0]
    num = float(find_square.split()[0].replace(",","."))
    return num
    
df["Площадь, м2"] = df.title.map(find_square_flat).astype(np.int32)

In [24]:
def find_flat_floor_and_total_floors(x: str, type_floor) -> tuple:
    """
    Получаем этаж квартиры и количество этажей в доме
    
    :param type_floor: 0 - этаж квартиры, 1 - количество этажей в доме
    """
    pattern = r", \d+/\d+ этаж"
    find_floors = re.findall(pattern, x)[0]
    floors = tuple(map(int,find_floors.split()[1].split("/")))
    return floors[type_floor]

df["Этаж"] = df.title.apply(find_flat_floor_and_total_floors, type_floor=0).astype(np.int32)
df["Всего этажей"] = df.title.apply(find_flat_floor_and_total_floors, type_floor=1).astype(np.int32)

# Мы рассматриваем только многоэтажные дома
df = df[df["Всего этажей"] > 1]

In [25]:
def find_district_and_street(x: str, type_place) -> tuple:
    """
    Получаем район и улицу
    
    :param type_place: 0 - район, 1 - улица
    """
    find_place = x.split(",")
    place = (find_place[2].strip(), "".join(find_place[-2:]).strip())
    return place[type_place]

df["Район"] = df.address.apply(find_district_and_street, type_place=0)
df["Адрес"] = df.address.apply(find_district_and_street, type_place=1)

In [26]:
df["Цена за квадрат, ₽/м²"] = df.price_per_square_meterice.map(lambda x: int(''.join(re.findall(r"\d+",x)))).astype(np.int32)

In [27]:
df["Цена, ₽"] = df.loc[df.price.str.contains("₽"),"price"].map(lambda x: int(''.join(re.findall(r"\d+",x))))

In [28]:
# При сборе данных некоторые ячейки в колонке price заполнились не ценами на квартиру
# Но у нас есть данные о цене за квадрат и площади, поэтому мы можем высчитать цену за квартиру самостоятельно
fail_price_df = df.loc[~df.price.str.contains("₽"),["price","Цена за квадрат, ₽/м²","Площадь, м2"]]
df.loc[~df.price.str.contains("₽"),"Цена, ₽"] = (fail_price_df.iloc[:,1] * fail_price_df.iloc[:,2])

In [29]:
df["Цена, ₽"] = df["Цена, ₽"].astype(np.int32)
df["Продавец"] = df["seller"]

In [30]:
processed_df = df[["Продавец", "Район", "Адрес", "Площадь, м2","Этаж", "Всего этажей","Цена, ₽", "Цена за квадрат, ₽/м²"]]
processed_df.head()

,Продавец,Район,Адрес,"Площадь, м2",Этаж,Всего этажей,"Цена, ₽","Цена за квадрат, ₽/м²"
0,Обмен.рф - Брусника Обмен,Кировский,улица Ватутина 24/1,66,7,10,6500000,97305
1,Обмен.рф - Брусника Обмен,Кировский,улица Крупской 12,71,1,10,7700000,108298
2,КВАРТИРНОЕ РЕШЕНИЕ,Советский,улица Лаптева 3,61,1,9,5890000,96557
3,ID 70295415,Октябрьский,улица Свободы 45,62,9,9,4495000,72383
4,ID 79765445,Ленинский,проспект Карла Маркса 40,76,3,10,12311392,161992


In [31]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1424 entries, 0 to 1526
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Продавец               1424 non-null   object
 1   Район                  1424 non-null   object
 2   Адрес                  1424 non-null   object
 3   Площадь, м2            1424 non-null   int32 
 4   Этаж                   1424 non-null   int32 
 5   Всего этажей           1424 non-null   int32 
 6   Цена, ₽                1424 non-null   int32 
 7   Цена за квадрат, ₽/м²  1424 non-null   int32 
dtypes: int32(5), object(3)
memory usage: 72.3+ KB


In [32]:
processed_df.to_csv("../data/processed_data_about_apartments.csv", sep=";",index=False)